# Notebook 03 — Whisper : Inférence et Évaluation ASR sur les Langues Africaines

## Introduction

Dans le notebook précédent, nous avons implémenté les composants de **wav2vec 2.0**, une architecture **encodeur-seul** qui apprend des représentations audio par apprentissage auto-supervisé contrastif.

**Whisper** (Radford et al., 2022) adopte une approche radicalement différente :

| | wav2vec 2.0 | Whisper |
|---|---|---|
| **Architecture** | Encodeur seul | Encodeur-Décodeur |
| **Pré-entraînement** | Auto-supervisé (contrastif) | Supervisé faiblement (680k heures) |
| **Entrée** | Waveform brute → Feature Encoder CNN | Log-Mel spectrogram (80/128 canaux) |
| **Sortie** | Représentations latentes | Texte (tokens) |
| **Décodage** | Nécessite un head CTC/seq2seq | Auto-régressif avec tokens spéciaux |
| **Multitâche** | Non | Oui (transcription, traduction, détection de langue) |

Whisper utilise un **Transformer encodeur-décodeur** classique, conditionné par des **tokens spéciaux** qui contrôlent la tâche (transcription vs traduction), la langue, et les timestamps.

Ce notebook couvre :
1. Chargement d'un modèle Whisper pré-entraîné via HuggingFace
2. Inférence sur un échantillon audio
3. Implémentation from scratch des métriques WER/CER (distance de Levenshtein)
4. Comparaison des performances sur langues africaines vs langues à haute ressource

In [13]:
import sys
from pathlib import Path

# Ajouter src/ au path
src_path = str(Path("../../src").resolve())
if src_path not in sys.path:
    sys.path.insert(0, src_path)

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['figure.dpi'] = 100

## Cell 2 — Chargement du modèle Whisper pré-entraîné

Nous utilisons la bibliothèque `transformers` de HuggingFace pour charger Whisper.

Whisper est disponible en plusieurs tailles :
- `whisper-tiny` (39M paramètres)
- `whisper-base` (74M)
- `whisper-small` (244M)
- `whisper-medium` (769M)
- `whisper-large-v3` (1.5B)

Pour ce notebook, nous utilisons `whisper-small` comme bon compromis taille/performance.

In [ ]:
%%capture
!pip install transformers

In [11]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Charger le processeur (tokenizer + feature extractor) et le modèle
model_name = "openai/whisper-small"

try:
    processor = WhisperProcessor.from_pretrained(model_name)
    model = WhisperForConditionalGeneration.from_pretrained(model_name)
    model.eval()
    print(f"Modèle chargé : {model_name}")
    print(f"Nombre de paramètres : {sum(p.numel() for p in model.parameters()):,}")
    print(f"\nArchitecture :")
    print(f"  Encodeur : {model.config.encoder_layers} couches, d_model={model.config.d_model}")
    print(f"  Décodeur : {model.config.decoder_layers} couches")
    print(f"  Attention heads : {model.config.encoder_attention_heads}")
    print(f"  Vocabulaire : {model.config.vocab_size} tokens")
except OSError as e:
    print(f"Erreur de chargement (vérifiez votre connexion) : {e}")
    print("Vous pouvez télécharger le modèle manuellement avec :")
    print(f"  huggingface-cli download {model_name}")

Loading weights:   0%|          | 0/479 [00:00<?, ?it/s]

Modèle chargé : openai/whisper-small
Nombre de paramètres : 241,734,912

Architecture :
  Encodeur : 12 couches, d_model=768
  Décodeur : 12 couches
  Attention heads : 12
  Vocabulaire : 51865 tokens


## Cell 3 — Inférence sur un échantillon audio

Le pipeline Whisper :
1. **Prétraitement** : Audio → Log-Mel spectrogram (80 canaux, fenêtres 25ms, pas 10ms)
2. **Encodeur** : Log-Mel → représentations contextuelles
3. **Décodeur** : Génération auto-régressive de tokens avec Cross-Attention

Les **tokens spéciaux** contrôlent le comportement :
```
<|startoftranscript|> <|fr|> <|transcribe|> <|notimestamps|> ... texte ... <|endoftext|>
```

In [6]:
!apt update
!apt install -y ffmpeg
#!pip install torchcodec

Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [ ]:
from datasets import load_dataset, Audio as adi
from IPython.display import Audio

dataset = load_dataset("FrancoisT/french_story_teller", split="test",  streaming=True)

#Audio(audio_sample["array"], rate=audio_sample["sampling_rate"]) 

print(f"Fréquence d'échantillonnage originale : {dataset[0]['audio']['sampling_rate']} Hz")

dataset = dataset.cast_column("audio", adi(sampling_rate=16000))

print(f"Fréquence d'échantillonnage après recast : {dataset[0]['audio']['sampling_rate']} Hz")

audio_sample = next(iter(dataset))["audio"]

Audio(audio_sample["array"], rate=audio_sample["sampling_rate"])             

Resolving data files:   0%|          | 0/56 [00:00<?, ?it/s]

Fréquence d'échantillonnage originale : <datasets.iterable_dataset.IterableColumn object at 0x79e5409a15b0> Hz
Fréquence d'échantillonnage après recast : <datasets.iterable_dataset.IterableColumn object at 0x79e5409a1a00> Hz


In [17]:
from audio.preprocessing import load_audio
# Charger l'audio d'exemple
audio_path = "../../data/raw/sample_audio.wav"
waveform, sr = load_audio(audio_path, target_sr=16000)

waveform = audio_sample["array"]
sr = audio_sample["sampling_rate"]





print(f"Audio chargé : {len(waveform)/sr:.2f}s à {sr} Hz")

# Prétraitement : le processor convertit l'audio en Log-Mel spectrogram
input_features = processor(
    waveform, 
    sampling_rate=sr, 
    return_tensors="pt"
).input_features

print(f"Shape Log-Mel : {input_features.shape}")  # (1, 80, 3000) pour 30s max

# Inférence : génération auto-régressive
with torch.no_grad():
    # forced_decoder_ids force la langue et la tâche
    predicted_ids = model.generate(
        input_features,
        language="fr",
        task="transcribe",
    )

# Décodage des tokens en texte
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
print(f"\nTranscription : {transcription}")

# Afficher les tokens spéciaux
tokens_with_special = processor.batch_decode(predicted_ids, skip_special_tokens=False)[0]
print(f"\nTokens avec spéciaux : {tokens_with_special}")

# Décomposer les token IDs
print(f"\nToken IDs ({len(predicted_ids[0])} tokens) :")
for i, tid in enumerate(predicted_ids[0][:10]):
    token_str = processor.tokenizer.decode([tid])
    print(f"  [{i}] ID={tid.item():5d} → '{token_str}'")
if len(predicted_ids[0]) > 10:
    print(f"  ... ({len(predicted_ids[0]) - 10} tokens restants)")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Audio chargé : 6.78s à 16000 Hz
Shape Log-Mel : torch.Size([1, 80, 3000])


A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensLogitsProcessor'> to see related `.generate()` flags.
A custom logits processor of type <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> has been passed to `.generate()`, but it was also created in `.generate()`, given its parameterization. The custom <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> will take precedence. Please check the docstring of <class 'transformers.generation.logits_process.SuppressTokensAtBeginLogitsProcessor'> to see related `.generate()` flags.



Transcription :  Nietzsche nous encourage à faire face à nos peurs de manière consciente et courageuse, sans chercher à les éviter ou à les nier.

Tokens avec spéciaux :  Nietzsche nous encourage à faire face à nos peurs de manière consciente et courageuse, sans chercher à les éviter ou à les nier.

Token IDs (32 tokens) :
  [0] ID=36583 → ' Niet'
  [1] ID=   89 → 'z'
  [2] ID=12287 → 'sche'
  [3] ID= 4666 → ' nous'
  [4] ID= 5373 → ' encourage'
  [5] ID= 1531 → ' à'
  [6] ID= 4865 → ' faire'
  [7] ID= 1851 → ' face'
  [8] ID= 1531 → ' à'
  [9] ID= 3269 → ' nos'
  ... (22 tokens restants)


## Cell 4 — IMPLÉMENTATION FROM SCRATCH : Distance de Levenshtein et WER/CER

Pour évaluer un système ASR, on compare la transcription produite (hypothèse) au texte de référence.

### Distance d'édition de Levenshtein

La distance de Levenshtein entre deux séquences est le nombre minimal d'opérations élémentaires pour transformer l'une en l'autre :
- **Insertion** d'un élément
- **Suppression** d'un élément  
- **Substitution** d'un élément par un autre

On la calcule par **programmation dynamique** avec une matrice `(n+1) × (m+1)` :

$$dp[i][j] = \begin{cases} i & \text{si } j = 0 \\ j & \text{si } i = 0 \\ dp[i-1][j-1] & \text{si } ref[i] = hyp[j] \\ 1 + \min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1]) & \text{sinon} \end{cases}$$

### Métriques
- **WER** (Word Error Rate) = `levenshtein(ref_mots, hyp_mots) / len(ref_mots)`
- **CER** (Character Error Rate) = `levenshtein(ref_chars, hyp_chars) / len(ref_chars)`

In [ ]:
from audio.metrics import (
    levenshtein_distance_from_scratch,
    compute_wer_from_scratch,
    compute_cer_from_scratch,
)

# Exemple concret
reference = "le chat est assis sur le tapis"
hypothesis = "le chat assis sur le tapi"

# --- Distance de Levenshtein au niveau des mots ---
ref_words = reference.split()
hyp_words = hypothesis.split()
print(f"Référence  : {ref_words}")
print(f"Hypothèse  : {hyp_words}")

dist_words = levenshtein_distance_from_scratch(ref_words, hyp_words)
print(f"\nDistance de Levenshtein (mots) : {dist_words}")
print(f"  → 'est' supprimé, 'tapis'→'tapi' substitué = 2 opérations")

# --- WER ---
wer = compute_wer_from_scratch(reference, hypothesis)
print(f"\nWER = {dist_words} / {len(ref_words)} = {wer:.4f} ({wer*100:.1f}%)")

# --- CER ---
cer = compute_cer_from_scratch(reference, hypothesis)
ref_chars = list(reference)
hyp_chars = list(hypothesis)
dist_chars = levenshtein_distance_from_scratch(ref_chars, hyp_chars)
print(f"CER = {dist_chars} / {len(ref_chars)} = {cer:.4f} ({cer*100:.1f}%)")

# --- Visualisation de la matrice DP ---
print("\n--- Matrice de programmation dynamique (mots) ---")
n, m = len(ref_words), len(hyp_words)
dp = [[0] * (m + 1) for _ in range(n + 1)]
for i in range(n + 1):
    dp[i][0] = i
for j in range(m + 1):
    dp[0][j] = j
for i in range(1, n + 1):
    for j in range(1, m + 1):
        if ref_words[i-1] == hyp_words[j-1]:
            dp[i][j] = dp[i-1][j-1]
        else:
            dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])

# Affichage formaté
header = [''] + ['∅'] + hyp_words
print(f"{'':>10s}", end='')
for h in ['∅'] + hyp_words:
    print(f"{h:>10s}", end='')
print()
for i, row in enumerate(dp):
    label = '∅' if i == 0 else ref_words[i-1]
    print(f"{label:>10s}", end='')
    for val in row:
        print(f"{val:>10d}", end='')
    print()

## Cell 5 — IMPLÉMENTATION JIWER : Calcul optimisé WER/CER

La bibliothèque **jiwer** fournit les mêmes métriques en 2 lignes, avec des optimisations et des transformations de normalisation intégrées.

In [ ]:
from audio.metrics import compute_wer, compute_cer

reference = "le chat est assis sur le tapis"
hypothesis = "le chat assis sur le tapi"

# Calcul via jiwer
wer_jiwer = compute_wer(reference, hypothesis)
cer_jiwer = compute_cer(reference, hypothesis)

# Calcul from scratch pour comparaison
wer_scratch = compute_wer_from_scratch(reference, hypothesis)
cer_scratch = compute_cer_from_scratch(reference, hypothesis)

print("Comparaison des implémentations :")
print(f"{'Métrique':<10s} {'From Scratch':>15s} {'jiwer':>15s} {'Match':>10s}")
print("-" * 55)
print(f"{'WER':<10s} {wer_scratch:>15.4f} {wer_jiwer:>15.4f} {'✓' if abs(wer_scratch - wer_jiwer) < 1e-6 else '✗':>10s}")
print(f"{'CER':<10s} {cer_scratch:>15.4f} {cer_jiwer:>15.4f} {'✓' if abs(cer_scratch - cer_jiwer) < 1e-6 else '✗':>10s}")

# Test d'identité
print(f"\nTest d'identité (ref == hyp) :")
print(f"  WER = {compute_wer(reference, reference):.4f} (attendu : 0.0)")
print(f"  CER = {compute_cer(reference, reference):.4f} (attendu : 0.0)")

## Cell 6 — Tableau comparatif : Performances Whisper sur langues africaines

Whisper a été entraîné principalement sur des données en anglais (~60%) et en langues européennes. Les performances sur les langues africaines sont significativement plus faibles, illustrant le défi des **langues peu dotées** (low-resource languages).

Les chiffres ci-dessous sont issus de la littérature (Radford et al. 2022, Whisper paper) et de benchmarks communautaires (OpenASR Leaderboard, Fleurs).

In [ ]:
import pandas as pd

# Données de performance Whisper (approximatives, basées sur Fleurs benchmark)
# Sources : Radford et al. 2022, OpenASR Leaderboard
performance_data = {
    "Langue": [
        "Anglais", "Français",
        "Swahili", "Wolof", "Yoruba",
    ],
    "Code ISO": ["en", "fr", "sw", "wo", "yo"],
    "Famille": [
        "Germanique", "Romane",
        "Bantoue", "Atlantique", "Volta-Niger",
    ],
    "Heures données (approx.)": [
        "~438k", "~85k",
        "~2k", "<100", "<500",
    ],
    "WER Whisper-small (%)": [
        8.5, 12.3,
        28.7, 62.4, 48.1,
    ],
    "WER Whisper-large-v3 (%)": [
        4.2, 7.1,
        18.5, 45.2, 32.6,
    ],
    "Défis spécifiques": [
        "Référence (haute ressource)",
        "Référence (haute ressource)",
        "Morphologie agglutinante, code-switching",
        "Tonalité, très peu de données",
        "Tonalité (3 niveaux), diacritiques",
    ],
}

df = pd.DataFrame(performance_data)
print("=" * 100)
print("TABLEAU COMPARATIF — Performances Whisper par langue")
print("=" * 100)
print(df.to_string(index=False))
print("\nSources : Radford et al. 2022, Fleurs benchmark, OpenASR Leaderboard")
print("Note : Les valeurs sont approximatives et varient selon le benchmark utilisé.")

# Visualisation
fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(df))
width = 0.35

bars1 = ax.bar(x - width/2, df["WER Whisper-small (%)"], width, 
               label="Whisper-small", color="#4C72B0", alpha=0.8)
bars2 = ax.bar(x + width/2, df["WER Whisper-large-v3 (%)"], width,
               label="Whisper-large-v3", color="#DD8452", alpha=0.8)

ax.set_ylabel("WER (%)")
ax.set_title("Word Error Rate par langue — Whisper small vs large-v3")
ax.set_xticks(x)
ax.set_xticklabels(df["Langue"])
ax.legend()
ax.axhline(y=20, color='gray', linestyle='--', alpha=0.5, label='Seuil usable (20%)')

# Ajouter les valeurs sur les barres
for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
            f'{bar.get_height():.1f}', ha='center', va='bottom', fontsize=9)
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
            f'{bar.get_height():.1f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Ratio de dégradation
print("\nRatio de dégradation par rapport à l'anglais (Whisper-small) :")
en_wer = df[df["Langue"] == "Anglais"]["WER Whisper-small (%)"].values[0]
for _, row in df.iterrows():
    ratio = row["WER Whisper-small (%)"] / en_wer
    print(f"  {row['Langue']:>10s} : WER = {row['WER Whisper-small (%)']:5.1f}% → ×{ratio:.1f} vs anglais")

## Cell 7 — Limitations et Hallucinations de Whisper

### Problèmes connus

1. **Hallucinations sur les langues peu dotées**
   - Whisper peut générer du texte fluide mais complètement inventé quand l'audio est dans une langue peu représentée
   - Le modèle « hallucine » en produisant du texte dans une langue à haute ressource (souvent l'anglais)
   - Ce phénomène est particulièrement fréquent pour le Wolof, le Yoruba et d'autres langues africaines

2. **Biais de données d'entraînement**
   - ~60% des données sont en anglais
   - Les langues africaines représentent < 1% du corpus total
   - Les accents et variétés dialectales sont sous-représentés

3. **Défis linguistiques spécifiques**
   - **Tonalité** : Whisper ne capture pas bien les distinctions tonales (Yoruba : owó=argent vs owo=respect)
   - **Morphologie agglutinante** : Les mots longs en Swahili/Zoulou sont mal segmentés par le tokenizer BPE
   - **Code-switching** : L'alternance Swahili-Anglais ou Yoruba-Anglais perturbe la détection de langue

4. **Heuristiques anti-hallucination**
   - **Temperature Fallback** : Si la log-probabilité moyenne est trop basse, on augmente la température et on ré-échantillonne
   - **Compression Gzip** : Si le ratio de compression du texte généré est anormalement élevé, c'est un signe de répétition/hallucination
   - **Seuil de log-probabilité** : Les segments avec une confiance trop faible sont marqués comme incertains

### Pistes d'amélioration

- **Fine-tuning** sur des données spécifiques à la langue cible (même quelques heures aident)
- **Adaptateurs** (voir Notebook 04) : Ajouter des modules légers sans modifier le modèle de base
- **MMS** (Meta) : Architecture spécialement conçue pour 1100+ langues avec des adaptateurs par langue
- **Augmentation de données** : Synthèse vocale (TTS) pour générer des données d'entraînement supplémentaires

In [ ]:
# Résumé des points clés
print("=" * 70)
print("RÉSUMÉ — Points clés du Notebook 03")
print("=" * 70)
print()
print("1. Whisper = Transformer encodeur-décodeur supervisé (680k heures)")
print("2. Entrée : Log-Mel spectrogram → Sortie : tokens texte")
print("3. Tokens spéciaux contrôlent langue, tâche, timestamps")
print("4. WER = distance d'édition (mots) / nombre de mots référence")
print("5. CER = distance d'édition (caractères) / nombre de caractères")
print("6. Performance dégradée sur langues africaines (×3 à ×7 vs anglais)")
print("7. Hallucinations fréquentes sur langues peu dotées")
print("8. Solutions : fine-tuning, adaptateurs (MMS), augmentation de données")
print()
print("→ Notebook suivant : Implémentation d'adaptateurs pour langues africaines")